<a href="https://colab.research.google.com/github/MitjaGo/YT/blob/main/You_tube_Downloader_MP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================
# STEP 1: Install dependencies
# ==========================
!pip install -q yt-dlp pydub mutagen ffmpeg-python requests tqdm

# ==========================
# STEP 2: Import libraries
# ==========================
import os
import json
import subprocess
import requests
import shutil
from pydub import AudioSegment, silence
from mutagen.easyid3 import EasyID3
from mutagen.id3 import ID3, APIC
from google.colab import files
from tqdm.notebook import tqdm

# ==========================
# STEP 3: Get YouTube URL
# ==========================
youtube_url = input("📥 Paste the YouTube URL here and press ENTER: ").strip()

# ==========================
# STEP 4: Create directories
# ==========================
os.makedirs("downloads", exist_ok=True)
os.makedirs("sections", exist_ok=True)

# ==========================
# STEP 5: Download audio
# ==========================
output_path = "downloads/youtube_audio.%(ext)s"
print("⬇️ Downloading audio...")
!yt-dlp -x --audio-format mp3 -o "{output_path}" "{youtube_url}" --no-playlist -q

# Find downloaded file
downloaded_files = os.listdir("downloads")
audio_file = [f for f in downloaded_files if f.endswith(".mp3")][0]
audio_path = os.path.join("downloads", audio_file)
print("✅ Audio downloaded:", audio_path)

# Load audio
audio = AudioSegment.from_file(audio_path)

# ==========================
# STEP 6: Get video info and thumbnail
# ==========================
print("⬇️ Fetching video info and thumbnail...")
result = subprocess.run(['yt-dlp', '-J', youtube_url], capture_output=True, text=True)
info = json.loads(result.stdout)
video_title = info['title']
video_author = info.get('uploader', 'Unknown')
thumbnail_url = info['thumbnail']

# Download thumbnail
thumb_path = "downloads/thumbnail.jpg"
thumb_data = requests.get(thumbnail_url).content
with open(thumb_path, "wb") as f:
    f.write(thumb_data)
print("🖼️ Thumbnail downloaded.")

# ==========================
# STEP 7: Split audio by silence
# ==========================
MIN_SILENCE_LEN = 1000  # 1 second
SILENCE_THRESH = audio.dBFS - 16  # adjust if too sensitive
KEEP_SILENCE = 500  # ms

print("✂️ Splitting audio by silence...")
chunks = silence.split_on_silence(
    audio,
    min_silence_len=MIN_SILENCE_LEN,
    silence_thresh=SILENCE_THRESH,
    keep_silence=KEEP_SILENCE
)

# If no silence detected, export full audio as single chunk
if not chunks:
    print("⚠️ No silence detected — using full audio as single section.")
    chunks = [audio]

# ==========================
# STEP 8: Export sections with metadata
# ==========================
sections = []

for i, chunk in enumerate(tqdm(chunks, desc="Processing sections")):
    temp_filename = f"sections/section_{i+1}.mp3"
    chunk.export(temp_filename, format="mp3")

    section_title = f"{video_title} - Part {i+1}"

    # ID3 metadata
    audiofile = EasyID3(temp_filename)
    audiofile['title'] = section_title
    audiofile['artist'] = video_author
    audiofile['album'] = video_title
    audiofile['tracknumber'] = str(i+1)
    audiofile.save()

    # Embed thumbnail
    tags = ID3(temp_filename)
    with open(thumb_path, 'rb') as img:
        tags.add(APIC(
            encoding=3,
            mime='image/jpeg',
            type=3,
            desc='Cover',
            data=img.read()
        ))
    tags.save()

    # Rename file safely
    safe_title = audiofile['title'][0].replace("/", "_").replace("\\", "_")
    safe_artist = audiofile['artist'][0].replace("/", "_").replace("\\", "_")
    new_filename = f"sections/{safe_title} - {safe_artist}.mp3"
    os.rename(temp_filename, new_filename)

    sections.append(new_filename)

print(f"💾 {len(sections)} sections exported successfully!")

# ==========================
# STEP 9: Create summary file
# ==========================
with open("sections_info.txt", "w") as f:
    for i, s in enumerate(sections, 1):
        f.write(f"{i}. {s}\n")
print("📝 Summary file created: sections_info.txt")

# ==========================
# STEP 10: Zip all sections
# ==========================
shutil.make_archive("sections", "zip", "sections")
print("📦 All sections zipped as sections.zip")

# Download summary and zip
files.download("sections_info.txt")
files.download("sections.zip")
